In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from dask.distributed import Client, wait
from dask import array as darr

from pathlib import Path

import hextraj

In [2]:
import cartopy
from matplotlib import pyplot as plt

## Parameters

We need to specify a few things for the trajectory data (location, chunks).

We also need to specify the hex projectoin. We choose a central position well withing the NWE shelf region (3W, 54.5N) and a hex size (radius) of 10 km

In [3]:
# parameters

data_path = "../../../output_data/010_lagrangian_experiment/"
_chunks_traj = 20_000
_chunks_obs = -1

chunks_traj = 20_000
chunks_obs = -1

# ~ center of NWE Shelf dataset
lon_origin = -3.0
lat_origin = 54.5

hex_size_meters = 10_000  # radius of hexes

max_land_allowed = 0

## Open the trajectory data

In [4]:
data_files = {y: sorted(
              f for f in Path(data_path).glob(f"*{y:4d}-05-15*.nc")
              if "start-depth-meters-010" in f.name
              ) 
              for y in [2019, 2020, 2021, 2022]
}
print(data_files[2019][0])
print(len(data_files[2019]))
print(len(data_files[2020]))
print(len(data_files[2021]))
print(len(data_files[2022]))
print(data_files[2019][-1])

../../../output_data/010_lagrangian_experiment/north-sea_oysters_start-ref-2019-05-15T00:00:00_start-offset-000_RT-0028_N-000100000_seed-576251074_start-depth-meters-010.nc
122
122
122
73
../../../output_data/010_lagrangian_experiment/north-sea_oysters_start-ref-2019-05-15T00:00:00_start-offset-121_RT-0028_N-000100000_seed-723534670_start-depth-meters-010.nc


In [5]:
from tqdm import tqdm

In [6]:
data_files_valid = {}
for y in data_files.keys():
    _valid_files = []
    for f in tqdm(data_files[y]):
        try:
            _ = xr.open_dataset(f)
            _valid_files.append(f)
        except:
            pass
    data_files_valid[y] = _valid_files

100%|██████████| 73/73 [00:15<00:00,  4.74it/s]


In [7]:
data_files = data_files_valid

In [8]:
ds = {
    y: xr.open_mfdataset(
        data_files[y],
        chunks={"traj": _chunks_traj, "obs": _chunks_obs},
        combine="nested",
        concat_dim="offset",
        # not sure what's wrong here, but we cannot write
        # zarr store in parallel if we decode:
        decode_cf=False,
    )
    for y in data_files.keys()
}
for y in ds.keys():
    ds[y] = ds[y].assign_coords(offset=("offset", list(range(ds[y].dims["offset"]))))
    
ds = xr.concat(ds.values(), dim="year")

display(ds)
print(ds.nbytes / 1e9, "GiB")

/tmp/ipykernel_290380/18525031.py:14: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  ds[y] = ds[y].assign_coords(offset=("offset", list(range(ds[y].dims["offset"]))))


<xarray.Dataset> Size: 1TB
Dimensions:      (year: 4, offset: 122, traj: 100000, obs: 673)
Coordinates:
  * offset       (offset) int64 976B 0 1 2 3 4 5 6 ... 116 117 118 119 120 121
Dimensions without coordinates: year, traj, obs
Data variables:
    time         (year, offset, traj, obs) float64 263GB dask.array<chunksize=(1, 1, 20000, 673), meta=np.ndarray>
    trajectory   (year, offset, traj, obs) float64 263GB dask.array<chunksize=(1, 1, 20000, 673), meta=np.ndarray>
    lon          (year, offset, traj, obs) float32 131GB dask.array<chunksize=(1, 1, 20000, 673), meta=np.ndarray>
    lat          (year, offset, traj, obs) float32 131GB dask.array<chunksize=(1, 1, 20000, 673), meta=np.ndarray>
    z            (year, offset, traj, obs) float32 131GB dask.array<chunksize=(1, 1, 20000, 673), meta=np.ndarray>
    temperature  (year, offset, traj, obs) float32 131GB dask.array<chunksize=(1, 1, 20000, 673), meta=np.ndarray>
    salinity     (year, offset, traj, obs) float32 131GB dask.array<chunksize=(1, 1, 20000, 673), meta=np.ndarray>
    land         (year, offset, traj, obs) float32 131GB dask.array<chunksize=(1, 1, 20000, 673), meta=np.ndarray>
Attributes:
    feature_type:           trajectory
    Conventions:            CF-1.6/CF-1.7
    ncei_template_version:  NCEI_NetCDF_Trajectory_Template_v2.0
    parcels_version:        2.3.1
    parcels_mesh:           spherical

1313.696000976 GiB


## Dask

In [9]:
#client = Client(n_workers=4, threads_per_worker=4, memory_limit=32e9, ip="0.0.0.0")
client = Client(scheduler_file="../../../scheduler.json")
#client.restart()
client

<Client: 'tcp://10.0.4.107:8786' processes=80 threads=160, memory=3.79 TiB>

In [10]:
ds.encoding = {}
for v in ds.data_vars:
    ds[v].encoding = {}

In [11]:
%%time

_ = ds.to_zarr(
    "../../../output_data/010_lagrangian_experiment_10m.zarr/",
    mode="w",
);

CPU times: user 8.33 s, sys: 469 ms, total: 8.8 s
Wall time: 8min 47s
